# Roll decay equation

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sympy as sp
#from rolldecayestimators.symbols import *
#import rolldecayestimators.equations as equations
from rolldecay.symbols import *
import rolldecay.equations as equations

import rolldecayestimators.direct_estimator as direct_estimator
import rolldecayestimators.direct_linear_estimator as direct_linear_estimator
import rolldecayestimators.direct_estimator_improved as direct_estimator_improved

from latex_helpers import pylatex_extenders
import latex_helpers
import os.path
from rigidbodysimulator.substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives
import dill

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

The roll motion can be written as [Himeno]:

In [ ]:
Math(vlatex(equations.roll_equation_himeno))

The equation express the roll moment (Nm) along a longitudinal axis though the centre of gravity.
Where $A_{44}$ is the virtual mass moment of intertia, $B_{44}$ is the roll damping moment and $C_{44}$ is the restoring moment. $M_{44}$ represents the external moment (usually moment from external waves).

The roll damping moment $B_{44}$ is the primary interest in this paper. The $B_{44}$ is determined using model scale roll decay tests. $B_{44}$ is determined using system identification, by finding the best fit to the following equation:    

In [ ]:
Math(vlatex(equations.roll_decay_equation_general_himeno))

The external moment is zero during a roll decay test, since there are no external forces present.

The $B_{44}$ can be expressed as a series expansion:  
$ B_{44} = B_1\cdot\dot{\phi} + B_2\cdot\dot{\phi}\left|\dot{\phi}\right| + B_3\cdot\dot{\phi}^3 + ...$

Truncating this series at the quadratic term gives a "quadratic damping model":

In [ ]:
B_1,B_2 = sp.symbols('B_1 B_2')

b44_quadratic_equation = sp.Eq(B_44,B_1*phi_dot + B_2*phi_dot*sp.Abs(phi_dot) )
Math(vlatex(b44_quadratic_equation))

Truncating this series at the linear term gives a "linear damping model":

In [ ]:
b44_quadratic_linear = sp.Eq(B_44,B_1*phi_dot)
Math(vlatex(b44_quadratic_linear))

Assuming quadratic damping the roll decay equation is written:

In [ ]:
eqs = [equations.roll_decay_equation_general_himeno,
            b44_quadratic_equation,
            ]

roll_decay_equation_himeno_quadratic =  equations.roll_decay_equation_general_himeno.subs(B_44,
                                                        sp.solve(b44_quadratic_equation,B_44)[0])

In [ ]:
Math(vlatex(roll_decay_equation_himeno_quadratic))

The restoring moment can be expressed using the $GZ$ curve:

In [ ]:
restoring_equation = sp.Eq(C_44,m*g*GZ)
Math(vlatex(restoring_equation))

where $m$ is the mass of the ship

It is common that the restoring moment is linearized using the meta centric height $GM$:

In [ ]:
restoring_equation_linear = sp.Eq(C_44,m*GM*phi)
Math(vlatex(restoring_equation_linear))

Introducing a helper coefficient $C$: 

In [ ]:
C = sp.Symbol('C')
C_equation = sp.Eq(C,C_44/phi)
Math(vlatex(C_equation))

In the case of linearized restoring moment $C$ can be written as:

In [ ]:
C_equation.subs(C_44,sp.solve(restoring_equation_linear,C_44)[0])

In [ ]:
roll_decay_equation_himeno_quadratic_c = roll_decay_equation_himeno_quadratic.subs(C_44,sp.solve(C_equation, C_44)[0])
Math(vlatex(roll_decay_equation_himeno_quadratic_c))

It is common to rewrite the roll decay equation by dividing with $A_{44}$ and conducting the following substitutions:

In [ ]:
zeta_equation = sp.Eq(2*zeta,B_1/A_44)
zeta_equation

In [ ]:
d_equation = sp.Eq(d,B_2/A_44)
d_equation

In [ ]:
omega0_equation = sp.Eq(omega0,sp.sqrt(C/A_44))
Math(vlatex(omega0_equation))

In [ ]:
eq = sp.Eq(roll_decay_equation_himeno_quadratic_c.lhs/A_44,0)

Commonly used quadratic roll decay equation:

In [ ]:
subs = [
    (B_1,sp.solve(zeta_equation,B_1)[0]),
    (B_2,sp.solve(d_equation,B_2)[0]),
    (C/A_44, sp.solve(omega0_equation,C/A_44)[0])
    
]

roll_decay_equation_quadratic = sp.Eq(sp.expand(eq.lhs).subs(subs),0)
roll_decay_equation_quadratic = sp.factor(roll_decay_equation_quadratic,phi_dot)

Math(vlatex(roll_decay_equation_quadratic))

And linear roll decay equation is obtained when $d=0$

In [ ]:
roll_decay_equation_linear = roll_decay_equation_quadratic.subs(d,0)

Math(vlatex(roll_decay_equation_linear))